#### 이탈 고객 예측

#### 데이터 전처리(결측치처리, 클래스 불균형 확인 및 처리)
#### 특성공학 - 피처선정 또는 피처생성
##### 베이스 모델선정
    - 분류모델 : LogisticRegression
    - 클래스 리포트, 과적합여부
##### 베이스 모델의 성능을 상회하는 모델과 하이퍼 파라메터 조합을 찾아서 성능을 출력

In [2]:
import pandas as pd
url = 'https://raw.githubusercontent.com/pia222sk20/python/refs/heads/main/data/tellecom.csv'
df = pd.read_csv(url)
